In [1]:
%load_ext autoreload
%autoreload 2

# Simple IEX Stock Fetcher

The IEX free message quota only gets one year of data for 200 stocks, so run this, fetcher stock data and save to disk every month.


## Test api

In [2]:
import iex
import key
iex.init(key.test_token, api='sandbox')
aapl = iex.Stock('AAPL')
aapl.chart('5d')

,change,changeOverTime,changePercent,close,high,label,low,open,uClose,uHigh,uLow,uOpen,uVolume,volume
date,,,,,,,,,,,,,,
2019-09-30,0.00,0.000000,0.0000,230.20,230.09,Sep 30,229.47,226.80,228.40,227.92,228.85,229.90,26841490,27506584
2019-10-01,0.62,0.002852,0.2858,234.56,228.73,Oct 1,234.70,226.03,226.30,237.90,230.50,235.12,37691578,37906031
2019-10-02,-5.68,-0.022798,-2.6024,219.46,229.68,Oct 2,227.08,223.94,219.07,234.70,223.84,234.00,36483928,36859828
2019-10-03,1.87,-0.014374,0.8615,221.36,228.88,Oct 3,224.74,226.18,230.40,230.14,217.64,228.59,31658842,30946087
2019-10-04,6.26,0.013779,2.8933,236.36,234.80,Oct 4,225.47,233.62,229.12,230.83,227.00,229.15,35594415,35560415


## Fetcher SPY stocks

In [3]:
import pandas as pd
spy = [x for x in pd.read_html('https://etfdailynews.com/etf/spy/', attrs={'id': 'etfs-that-own'})[0].Symbol.values.tolist() if isinstance(x, str)]
correction = {'PCLN':'BKNG', 'DWDP':'DD'}

In [4]:
import os, sys
chart_range = [(5,'break'), (10,'1m'), (45,'3m'), (90,'6m'), (250,'1y'), (99999,'max')]

In [ ]:
# iex.init(key.test_token, api='sandbox')
iex.init(key.token, api='cloud')

# save historical price to disk
for k in spy:
    if k in correction:
        k = correction[k]
    sys.stdout.write('\r{}   '.format(k))
    filename = "./daily/{}.csv".format(k)
    if os.path.exists(filename):
        # fetcher missing period
        df = pd.read_csv(filename, index_col='date', parse_dates=True)
        td = pd.Timedelta(df.index[-1] - pd.datetime.now())
        for day, param in chart_range:
            if td.days < day:
                if param != 'break':
                    df_append = iex.Stock(k).chart(param)
                    pd.concat([df, df_append]).to_csv(filename)
                break
    else:
        iex.Stock(k).chart('max').to_csv(filename)
    
